In [1]:
from glob import glob
import os
from PIL import Image
import json

In [2]:
os.listdir("../../../Munsu/Hanssem/cropData/train/PngImages/")

['255285_rug_1.png',
 '578528_closet_1.png',
 '606447_closet_1.png',
 '492367_closet_1.png',
 '210347_rug_1.png',
 '484203_closet_1.png',
 '429936_bed_1.png',
 '319574_bed_1.png',
 '470055_bed_1.png',
 '594302_bed_1.png',
 '480941_rug_1.png',
 '649138_closet_1.png',
 '400263_closet_2.png',
 '594588_closet_3.png',
 '543355_bed_1.png',
 '401708_closet_1.png',
 '544821_closet_2.png',
 '196410_rug_1.png',
 '603932_closet_2.png',
 '592129_bed_1.png',
 '649460_bed_1.png',
 '343681_closet_1.png',
 '589661_closet_1.png',
 '385252_closet_2.png',
 '370625_rug_1.png',
 '596900_closet_1.png',
 '385877_closet_1.png',
 '393658_bed_1.png',
 '614672_bed_1.png',
 '493480_closet_1.png',
 '358163_rug_1.png',
 '553615_bed_1.png',
 '482082_rug_1.png',
 '377661_closet_1.png',
 '550984_closet_1.png',
 '594372_closet_1.png',
 '385139_closet_1.png',
 '373544_closet_1.png',
 '385252_closet_5.png',
 '395752_bed_1.png',
 '326318_bed_1.png',
 '468168_bed_1.png',
 '543351_bed_1.png',
 '629149_bed_1.png',
 '369186_r

In [3]:
detected_train_formats = list()

for json_path in glob("../../../Munsu/Hanssem/cropData/train/JsonFiles/*.json"):
    with open(json_path, "rb") as f:
        jsonFile = json.load(f)
    detected_train_formats.append(jsonFile)

detected_train_formats[0]

{'objectID': 'feb3dbc6-e4fa-43a7-85f9-578e4de929b4',
 'location': {'xmin': 77, 'ymin': 232, 'xmax': 443, 'ymax': 423},
 'tag': 'bed',
 'objectImagePath': './cropData/train/PngImages/367651_bed_1.png',
 'IMG_URL': 'https://image.hanssem.com/hsimg//gds/550/367/367651_A1.jpg'}

In [4]:
len(detected_train_formats)

3398

In [5]:
image_path = [format["objectImagePath"] for format in detected_train_formats]
detected_images_train = [Image.open(path) for path in image_path]


In [6]:
from ImageRetrievalClass import ImageRetrievalClass


retrievalInstance = ImageRetrievalClass("MobileNetV3",True,False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [7]:
retrievalInstance.readTrainSet(detected_images_train)

Reading train images
train image shape = (256, 256, 3)


In [9]:
retrievalInstance.buildModel(shape_img=(256,256,3))

Loading MobileNetV3 pre-trained model...
input_shape_model = (256, 256, 3)
output_shape_model = (8, 8, 1024)


In [10]:
X_train = retrievalInstance.trainTransform()

Applying image transformer to training images...
 -> X_train.shape = (3398, 256, 256, 3)


In [11]:
import numpy as np
E_train = retrievalInstance.predictTrain(X_train)

Inferencing embeddings using pre-trained model...
 -> E_train.shape = (3398, 8, 8, 1024)


# 객체 각각의 npy 파일 저장

In [12]:
count = 0
for idx, format in enumerate(detected_train_formats):
    filename = format["objectImagePath"].split("/")[-1].split(".")[0]
    npyFilename ="./mobileNpy/"+filename+".npy"
    np.save(npyFilename, E_train[idx])
    count+=1
print(count)

3398


In [13]:
len(os.listdir("./mobileNpy/"))

3398

# Json 파일 저장

In [14]:
for idx, format in enumerate(detected_train_formats):
    filename = format["objectImagePath"].split("/")[-1].split(".")[0]
    npyFilename = "./mobileNpy/"+filename+".npy"
    jsonDict ={
        "objectID" : format["objectID"],
        "location" : format["location"],
        "tag":format["tag"],
        "objectImagePath" : format["objectImagePath"],
        "npyPath" : npyFilename,
        "IMG_URL" : format["IMG_URL"]
    }

    filename = "./mobileJson/"+filename+".json"

    with open(filename, "w") as outFile:
        json.dump(jsonDict, outFile, indent=2)